In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel
from dotenv import load_dotenv
import os

app_dir = os.path.join(os.getcwd(), 'app')
load_dotenv(os.path.join(app_dir, '.env'))

loader = DirectoryLoader('./data', glob="**/*.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=120,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(docs)

embedding_function = OpenAIEmbeddings()
model = ChatOpenAI()

db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()


In [ ]:
def format_docs(docs: list[Document]):
    return "\n".join(doc.page_content for doc in docs)


In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
result = rag_chain_with_source.invoke(input="Who is the owner of the restaurant")

In [ ]:
result

### Why is that approach bad?

You will always retrieve top-k documents and pass them to the model. 
No matter how relevant the documents are

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
docs = result["context"]
contents = [doc.page_content for doc in docs]

In [ ]:
docs

In [ ]:
pairs = []
for text in contents:
    pairs.append(["Who is the owner of the restaurant", text])

In [ ]:
scores = cross_encoder.predict(pairs)
scores

In [ ]:
scored_docs = zip(scores, docs)
sorted_docs = sorted(scored_docs, reverse=True)
sorted_docs

In [ ]:
reranked_docs = [doc for _, doc in sorted_docs][0:2]
reranked_docs

### Integrate that in LCEL

In [52]:
from sentence_transformers import CrossEncoder
from langchain_core.runnables import RunnableLambda

def rerank_documents(input_data):
    query = input_data["question"]
    docs = input_data["context"]

    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    contents = [doc.page_content for doc in docs]

    pairs = [(query, text) for text in contents]
    scores = cross_encoder.predict(pairs)

    scored_docs = zip(scores, docs)
    sorted_docs = sorted(scored_docs, key=lambda x: x[0], reverse=True)
    return [doc for _, doc in sorted_docs]

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=RunnableLambda(rerank_documents))
    | prompt
    | model
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [53]:
result = rag_chain_with_source.invoke(input="Who is the owner of the restaurant")
result

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


{'context': [Document(page_content="In the charming streets of Palermo, tucked away in a quaint alley, stood Chef Amico, a restaurant that was more than a mere eatery—it was a slice of Sicilian heaven. Founded by Amico, a chef whose name was synonymous with passion and creativity, the restaurant was a mosaic of his life’s journey through the flavors of Italy.\n\nChef Amico’s doors opened to a world where the aromas of garlic and olive oil were as welcoming as a warm embrace. The walls, adorned with photos of Amico’s travels and family recipes, spoke of a rich culinary heritage. The chatter and laughter of patrons filled the air, creating a symphony as delightful as the dishes served.\n\nOne evening, as the sun cast a golden glow over the city, a renowned food critic, Elena Rossi, stepped into Chef Amico. Her mission was to uncover the secret behind the restaurant's growing fame. She was greeted by Amico himself, whose eyes sparkled with the joy of a man who loved his work.\n\nElena was

### LLM-based Document Compressor

In [1]:
from langchain.prompts import PromptTemplate

DOCUMENT_EVALUATOR_PROMPT = PromptTemplate(
    input_variables=["document", "question"],
    template="""You are an AI language model assistant. Your task is to evaluate the provided document to determine if it is suited to answer the given user question. Assess the document for its relevance to the question, the completeness of information, and the accuracy of the content.

    Original question: {question}
    Document for Evaluation: {document}
    Evaluation Result: <<'True' if the document is suited to answer the question, 'False' if it is not>>

    Note: Conclude with a 'True' or 'False' based on your analysis of the document's relevance, completeness, and accuracy in relation to the question."""
)


In [ ]:
from langchain.schema import Document

documents = [
    Document(page_content="The owner is Guivanni"), Document(page_content="Pizza Salami costs 10$"),
    Document(page_content="We close the restaurant at 10p.m each day")
]

model = ChatOpenAI()
rephrase_chain = DOCUMENT_EVALUATOR_PROMPT | model | StrOutputParser()